In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

featuresdf=pd.read_csv('./csv/set4.csv', index_col=0)
featuresdf = featuresdf.reset_index()
set4fake = featuresdf.copy()
for column in ['acceleration_x','acceleration_y','acceleration_z', 'gyro_x', 'gyro_y', 'gyro_z','second', 'Speed']:
    justnullsdf = set4fake[set4fake[column].isnull()]
    bookingids = justnullsdf['bookingID'].unique() # get all the bookingids that have null values inside them
    rows = 0
    for i in range (len(bookingids)):
        bookingidx = set4fake[set4fake['bookingID']==bookingids[i]]
        if bookingidx[column].isnull().all().all():
            set4fake = set4fake.drop(set4fake[set4fake.bookingID == bookingids[i]].index)
            
uniqueValues = set4fake['bookingID'].unique()
uniqueValues = np.sort(uniqueValues)
bookingids=featuresdf['bookingID'].unique()


In [ ]:
print(set4fake.shape)
set4_1fake = set4fake[set4fake['bookingID'].isin(uniqueValues[0:1650])]
print(set4_1fake.shape)
set4_2fake = set4fake[set4fake['bookingID'].isin(uniqueValues[1650:2900])]
print(set4_2fake.shape)
set4_3fake = set4fake[set4fake['bookingID'].isin(uniqueValues[2900:])]
print(set4_3fake.shape)
print(set4_1fake.shape[0] + set4_2fake.shape[0] + set4_3fake.shape[0])

(188106, 12)
(66258, 12)
(50640, 12)
(71208, 12)
188106


## SET 4-1

In [ ]:
# THIS IS ONLY FOR SET 3_1fake acceleration_x
justnullsdf = set4_1fake[set4_1fake['acceleration_x'].isnull()]
bookingids = justnullsdf['bookingID'].unique() # get all the bookingids that have null values inside them
for bookingid in bookingids: #loop for each bookingid so that we are dealing with the null values bookingid by bookingid which is easier to work with
    bookingidx = set4_1fake[set4_1fake['bookingID']==bookingid] # making a datfarame with bookingID = x
    bookingidx = bookingidx.sort_values(by=['second']) # sorting the records by seconds so that it is easier to plot time against acceleration_x
    bookingidx = bookingidx.reset_index(drop=True) # reindexing the dataframe for easier coding
    cleanedSetbookingidx = bookingidx.dropna()
    if len(cleanedSetbookingidx) < 5:
        set4_1fake = set4_1fake.drop(set4_1fake[set4_1fake.bookingID == bookingid].index)
        continue
    std = np.std(bookingidx['acceleration_x']) # finding the standard deviation of acceleration_x
    nulldf = bookingidx[bookingidx['acceleration_x'].isnull()] # make dataframe of records with acceleration_x being a null value
    for ind in nulldf.index: #retrieve the index of the row with acceleration_x = NULL
        id = nulldf[nulldf.index.values == ind]['ID'] # get the value of 'ID' column for that specific null record
        previousrowindex = ind - 1 # this is to retrieve the index of the previous row 
        afterrowindex = ind + 1 # this is to retrieve the index of the next row  
        valueofpreviousrow = bookingidx.iloc[previousrowindex, 4] # this is to retrieve the acceleration of the previous row
        valueofafterrow = bookingidx[bookingidx.index.values == afterrowindex]['acceleration_x'] # this is to retrieve the acceleration of the next row
        print(f'bookingid: {bookingid}, index: {int(ind)}, id: {int(id)}')
        if ind == 0: # if the null record is the first record of xbookingid
            if bookingidx[bookingidx.index.values == afterrowindex]['acceleration_x'].empty or bookingidx[bookingidx.index.values == afterrowindex]['acceleration_x'].isnull().bool(): # if the next record is also a null value, this is for records that are consecutive AT THE START OF THE BOOKINGID
                print(f'the after value is {float(valueofafterrow)}')
                estimatedvalue = float(bookingidx[bookingidx.index.values == afterrowindex+1]['acceleration_x']) - std -std # to get estimatedvalue of record 1, record 3- std -std = record 1
            else: # if then next record IS NOT a null value
                print(f'the after value is {float(valueofafterrow)}')
                estimatedvalue = float(valueofafterrow) - std # the estimated value of record 1 = record 2 - std
        
        elif ind ==  len(bookingidx)-1: # if the null record is THE LAST VALUE OF THE BOOKINGID
            print(f'the previous value is {valueofpreviousrow}') 
            estimatedvalue = valueofpreviousrow + std # the estimated value of the last record = second last record + std
        
        elif bookingidx[bookingidx.index.values == afterrowindex]['acceleration_x'].empty or bookingidx[bookingidx.index.values == afterrowindex]['acceleration_x'].isnull().bool(): # if there are 2 consecutive null values and arent the first or last record
            print(f'the after value is {float(valueofafterrow)}')
            estimatedvalue = valueofpreviousrow + std # estimated value = previous record + std
        
        else:
            print(f'the previous value is {valueofpreviousrow} and the after value is {float(valueofafterrow)}')
            estimatedvalue = np.median([valueofpreviousrow,float(valueofafterrow)]) # the estimated value = median of previous record and next record
        print(f'estimated value is {estimatedvalue}')
        realindex =  set4_1fake[set4_1fake['ID'] == int(id)].index.values # to get the index of the record IN THE SET
        bookingidx.at[int(ind),'acceleration_x'] = estimatedvalue # to assign estimated value to the corresponding record IN THE CURRENT BOOKING DATAFRAME THE CODE IS WORKING WITH NOW
        set4_1fake.at[int(realindex),'acceleration_x'] = estimatedvalue # this is to assign estimated value to the corresponding record IN THE SET
        ss = set4_1fake[set4_1fake.index.values == int(realindex)]['acceleration_x'] # just for printing to debug
        zz = bookingidx[bookingidx.index.values == int(realindex)]['acceleration_x'] # just for printing to debug
        print(f'the value back in set4_1fake is now {ss}')
        print(f'the value back in bookingidx is now {zz}')
        print('-----------------------')

bookingid: 455266533505.0, index: 0, id: 7424
the after value is 1.7095122
estimated value is 1.0759601608036866
the value back in set4_1fake is now 186    1.07596
Name: acceleration_x, dtype: float64
the value back in bookingidx is now Series([], Name: acceleration_x, dtype: float64)
-----------------------
bookingid: 455266533505.0, index: 9, id: 891768
the previous value is 2.2314641 and the after value is -0.076616794
estimated value is 1.077423653
the value back in set4_1fake is now 22612    1.077424
Name: acceleration_x, dtype: float64
the value back in bookingidx is now Series([], Name: acceleration_x, dtype: float64)
-----------------------
bookingid: 403726925887.0, index: 1, id: 967764
the previous value is 1.4940275 and the after value is 0.5794145
estimated value is 1.036721
the value back in set4_1fake is now 24514    1.036721
Name: acceleration_x, dtype: float64
the value back in bookingidx is now Series([], Name: acceleration_x, dtype: float64)
-----------------------
bo

In [ ]:
set4_1fake.isnull().sum(axis = 0)

ID                   0
bookingID            0
Accuracy          1110
Bearing           1496
acceleration_x       0
acceleration_y    1591
acceleration_z     579
gyro_x            1213
gyro_y            1065
gyro_z            1263
second             914
Speed                0
dtype: int64

In [ ]:
# THIS IS ONLY FOR SET 3_1fake acceleration_y
justnullsdf = set4_1fake[set4_1fake['acceleration_y'].isnull()]
bookingids = justnullsdf['bookingID'].unique() # get all the bookingids that have null values inside them
for bookingid in bookingids: #loop for each bookingid so that we are dealing with the null values bookingid by bookingid which is easier to work with
    bookingidx = set4_1fake[set4_1fake['bookingID']==bookingid] # making a datfarame with bookingID = x
    bookingidx = bookingidx.sort_values(by=['second']) # sorting the records by seconds so that it is easier to plot time against acceleration_y
    bookingidx = bookingidx.reset_index(drop=True) # reindexing the dataframe for easier coding
    cleanedSetbookingidx = bookingidx.dropna()
    if len(cleanedSetbookingidx) < 5:
        set4_1fake = set4_1fake.drop(set4_1fake[set4_1fake.bookingID == bookingid].index)
        continue
    std = np.std(bookingidx['acceleration_y']) # finding the standard deviation of acceleration_y
    nulldf = bookingidx[bookingidx['acceleration_y'].isnull()] # make dataframe of records with acceleration_y being a null value
    for ind in nulldf.index: #retrieve the index of the row with acceleration_y = NULL
        id = nulldf[nulldf.index.values == ind]['ID'] # get the value of 'ID' column for that specific null record
        previousrowindex = ind - 1 # this is to retrieve the index of the previous row 
        afterrowindex = ind + 1 # this is to retrieve the index of the next row  
        valueofpreviousrow = bookingidx.iloc[previousrowindex, 5] # this is to retrieve the acceleration of the previous row
        valueofafterrow = bookingidx[bookingidx.index.values == afterrowindex]['acceleration_y'] # this is to retrieve the acceleration of the next row
        print(f'bookingid: {bookingid}, index: {int(ind)}, id: {int(id)}')
        if ind == 0: # if the null record is the first record of xbookingid
            if bookingidx[bookingidx.index.values == afterrowindex]['acceleration_y'].empty or bookingidx[bookingidx.index.values == afterrowindex]['acceleration_y'].isnull().bool(): # if the next record is also a null value, this is for records that are consecutive AT THE START OF THE BOOKINGID
                print(f'the after value is {float(valueofafterrow)}')
                estimatedvalue = float(bookingidx[bookingidx.index.values == afterrowindex+1]['acceleration_y']) - std -std # to get estimatedvalue of record 1, record 3- std -std = record 1
            else: # if then next record IS NOT a null value
                print(f'the after value is {float(valueofafterrow)}')
                estimatedvalue = float(valueofafterrow) - std # the estimated value of record 1 = record 2 - std
        
        elif ind ==  len(bookingidx)-1: # if the null record is THE LAST VALUE OF THE BOOKINGID
            print(f'the previous value is {valueofpreviousrow}') 
            estimatedvalue = valueofpreviousrow + std # the estimated value of the last record = second last record + std
        
        elif bookingidx[bookingidx.index.values == afterrowindex]['acceleration_y'].empty or bookingidx[bookingidx.index.values == afterrowindex]['acceleration_y'].isnull().bool(): # if there are 2 consecutive null values and arent the first or last record
            print(f'the after value is {float(valueofafterrow)}')
            estimatedvalue = valueofpreviousrow + std # estimated value = previous record + std
        
        else:
            print(f'the previous value is {valueofpreviousrow} and the after value is {float(valueofafterrow)}')
            estimatedvalue = np.median([valueofpreviousrow,float(valueofafterrow)]) # the estimated value = median of previous record and next record
        print(f'estimated value is {estimatedvalue}')
        realindex =  set4_1fake[set4_1fake['ID'] == int(id)].index.values # to get the index of the record IN THE SET
        bookingidx.at[int(ind),'acceleration_y'] = estimatedvalue # to assign estimated value to the corresponding record IN THE CURRENT BOOKING DATAFRAME THE CODE IS WORKING WITH NOW
        set4_1fake.at[int(realindex),'acceleration_y'] = estimatedvalue # this is to assign estimated value to the corresponding record IN THE SET
        ss = set4_1fake[set4_1fake.index.values == int(realindex)]['acceleration_y'] # just for printing to debug
        zz = bookingidx[bookingidx.index.values == int(realindex)]['acceleration_y'] # just for printing to debug
        print(f'the value back in set4_1fake is now {ss}')
        print(f'the value back in bookingidx is now {zz}')
        print('-----------------------')

bookingid: 446676598879.0, index: 11, id: 1175
the previous value is 8.933867999999999 and the after value is 9.365356
estimated value is 9.149612
the value back in set4_1fake is now 30    9.149612
Name: acceleration_y, dtype: float64
the value back in bookingidx is now 30    8.580826
Name: acceleration_y, dtype: float64
-----------------------
bookingid: 446676598879.0, index: 19, id: 2752081
the previous value is 9.600723 and the after value is 8.05127
estimated value is 8.8259965
the value back in set4_1fake is now 69551    8.825997
Name: acceleration_y, dtype: float64
the value back in bookingidx is now Series([], Name: acceleration_y, dtype: float64)
-----------------------
bookingid: 446676598879.0, index: 35, id: 3121933
the previous value is 9.698776 and the after value is 9.404587
estimated value is 9.5516815
the value back in set4_1fake is now 78986    9.551682
Name: acceleration_y, dtype: float64
the value back in bookingidx is now Series([], Name: acceleration_y, dtype: flo

In [ ]:
set4_1fake.isnull().sum(axis = 0)

ID                   0
bookingID            0
Accuracy          1108
Bearing           1495
acceleration_x       0
acceleration_y       0
acceleration_z     578
gyro_x            1208
gyro_y            1062
gyro_z            1259
second             910
Speed                0
dtype: int64

In [ ]:
# THIS IS ONLY FOR SET 3_1fake acceleration_z
justnullsdf = set4_1fake[set4_1fake['acceleration_z'].isnull()]
bookingids = justnullsdf['bookingID'].unique() # get all the bookingids that have null values inside them
for bookingid in bookingids: #loop for each bookingid so that we are dealing with the null values bookingid by bookingid which is easier to work with
    bookingidx = set4_1fake[set4_1fake['bookingID']==bookingid] # making a datfarame with bookingID = x
    bookingidx = bookingidx.sort_values(by=['second']) # sorting the records by seconds so that it is easier to plot time against acceleration_z
    bookingidx = bookingidx.reset_index(drop=True) # reindexing the dataframe for easier coding
    cleanedSetbookingidx = bookingidx.dropna()
    if len(cleanedSetbookingidx) < 5:
        set4_1fake = set4_1fake.drop(set4_1fake[set4_1fake.bookingID == bookingid].index)
        continue
    std = np.std(bookingidx['acceleration_z']) # finding the standard deviation of acceleration_z
    nulldf = bookingidx[bookingidx['acceleration_z'].isnull()] # make dataframe of records with acceleration_z being a null value
    for ind in nulldf.index: #retrieve the index of the row with acceleration_z = NULL
        id = nulldf[nulldf.index.values == ind]['ID'] # get the value of 'ID' column for that specific null record
        previousrowindex = ind - 1 # this is to retrieve the index of the previous row 
        afterrowindex = ind + 1 # this is to retrieve the index of the next row  
        valueofpreviousrow = bookingidx.iloc[previousrowindex, 6] # this is to retrieve the acceleration of the previous row
        valueofafterrow = bookingidx[bookingidx.index.values == afterrowindex]['acceleration_z'] # this is to retrieve the acceleration of the next row
        print(f'bookingid: {bookingid}, index: {int(ind)}, id: {int(id)}')
        if ind == 0: # if the null record is the first record of xbookingid
            if bookingidx[bookingidx.index.values == afterrowindex]['acceleration_z'].empty or bookingidx[bookingidx.index.values == afterrowindex]['acceleration_z'].isnull().bool(): # if the next record is also a null value, this is for records that are consecutive AT THE START OF THE BOOKINGID
                print(f'the after value is {float(valueofafterrow)}')
                estimatedvalue = float(bookingidx[bookingidx.index.values == afterrowindex+1]['acceleration_z']) - std -std # to get estimatedvalue of record 1, record 3- std -std = record 1
            else: # if then next record IS NOT a null value
                print(f'the after value is {float(valueofafterrow)}')
                estimatedvalue = float(valueofafterrow) - std # the estimated value of record 1 = record 2 - std
        
        elif ind ==  len(bookingidx)-1: # if the null record is THE LAST VALUE OF THE BOOKINGID
            print(f'the previous value is {valueofpreviousrow}') 
            estimatedvalue = valueofpreviousrow + std # the estimated value of the last record = second last record + std
        
        elif bookingidx[bookingidx.index.values == afterrowindex]['acceleration_z'].empty or bookingidx[bookingidx.index.values == afterrowindex]['acceleration_z'].isnull().bool(): # if there are 2 consecutive null values and arent the first or last record
            print(f'the after value is {float(valueofafterrow)}')
            estimatedvalue = valueofpreviousrow + std # estimated value = previous record + std
        
        else:
            print(f'the previous value is {valueofpreviousrow} and the after value is {float(valueofafterrow)}')
            estimatedvalue = np.median([valueofpreviousrow,float(valueofafterrow)]) # the estimated value = median of previous record and next record
        print(f'estimated value is {estimatedvalue}')
        realindex =  set4_1fake[set4_1fake['ID'] == int(id)].index.values # to get the index of the record IN THE SET
        bookingidx.at[int(ind),'acceleration_z'] = estimatedvalue # to assign estimated value to the corresponding record IN THE CURRENT BOOKING DATAFRAME THE CODE IS WORKING WITH NOW
        set4_1fake.at[int(realindex),'acceleration_z'] = estimatedvalue # this is to assign estimated value to the corresponding record IN THE SET
        ss = set4_1fake[set4_1fake.index.values == int(realindex)]['acceleration_z'] # just for printing to debug
        zz = bookingidx[bookingidx.index.values == int(realindex)]['acceleration_z'] # just for printing to debug
        print(f'the value back in set4_1fake is now {ss}')
        print(f'the value back in bookingidx is now {zz}')
        print('-----------------------')

bookingid: 429496729692.0, index: 103, id: 11670
the previous value is -2.8076904296875003 and the after value is -3.4663970947265628
estimated value is -3.1370437622070315
the value back in set4_1fake is now 302   -3.137044
Name: acceleration_z, dtype: float64
the value back in bookingidx is now 302   -2.480505
Name: acceleration_z, dtype: float64
-----------------------
bookingid: 429496729692.0, index: 198, id: 6481989
the previous value is -1.924380493164063 and the after value is 1.143951416015625
estimated value is -0.39021453857421895
the value back in set4_1fake is now 163358   -0.390215
Name: acceleration_z, dtype: float64
the value back in bookingidx is now Series([], Name: acceleration_z, dtype: float64)
-----------------------
bookingid: 429496729692.0, index: 258, id: 1676146
the previous value is -0.27335205078125 and the after value is -0.8720947265625001
estimated value is -0.572723388671875
the value back in set4_1fake is now 42488   -0.572723
Name: acceleration_z, dty

In [ ]:
set4_1fake.isnull().sum(axis = 0)

ID                   0
bookingID            0
Accuracy          1107
Bearing           1494
acceleration_x       0
acceleration_y       0
acceleration_z       0
gyro_x            1208
gyro_y            1059
gyro_z            1259
second             909
Speed                0
dtype: int64

In [ ]:
# THIS IS ONLY FOR SET 3_1fake gyro_x
justnullsdf = set4_1fake[set4_1fake['gyro_x'].isnull()]
bookingids = justnullsdf['bookingID'].unique() # get all the bookingids that have null values inside them
for bookingid in bookingids: #loop for each bookingid so that we are dealing with the null values bookingid by bookingid which is easier to work with
    bookingidx = set4_1fake[set4_1fake['bookingID']==bookingid] # making a datfarame with bookingID = x
    bookingidx = bookingidx.sort_values(by=['second']) # sorting the records by seconds so that it is easier to plot time against gyro_x
    bookingidx = bookingidx.reset_index(drop=True) # reindexing the dataframe for easier coding
    cleanedSetbookingidx = bookingidx.dropna()
    if len(cleanedSetbookingidx) < 5:
        set4_1fake = set4_1fake.drop(set4_1fake[set4_1fake.bookingID == bookingid].index)
        continue
    std = np.std(bookingidx['gyro_x']) # finding the standard deviation of gyro_x
    nulldf = bookingidx[bookingidx['gyro_x'].isnull()] # make dataframe of records with gyro_x being a null value
    for ind in nulldf.index: #retrieve the index of the row with gyro_x = NULL
        id = nulldf[nulldf.index.values == ind]['ID'] # get the value of 'ID' column for that specific null record
        previousrowindex = ind - 1 # this is to retrieve the index of the previous row 
        afterrowindex = ind + 1 # this is to retrieve the index of the next row  
        valueofpreviousrow = bookingidx.iloc[previousrowindex, 7] # this is to retrieve the acceleration of the previous row
        valueofafterrow = bookingidx[bookingidx.index.values == afterrowindex]['gyro_x'] # this is to retrieve the acceleration of the next row
        print(f'bookingid: {bookingid}, index: {int(ind)}, id: {int(id)}')
        if ind == 0: # if the null record is the first record of xbookingid
            if bookingidx[bookingidx.index.values == afterrowindex]['gyro_x'].empty or bookingidx[bookingidx.index.values == afterrowindex]['gyro_x'].isnull().bool(): # if the next record is also a null value, this is for records that are consecutive AT THE START OF THE BOOKINGID
                print(f'the after value is {float(valueofafterrow)}')
                estimatedvalue = float(bookingidx[bookingidx.index.values == afterrowindex+1]['gyro_x']) - std -std # to get estimatedvalue of record 1, record 3- std -std = record 1
            else: # if then next record IS NOT a null value
                print(f'the after value is {float(valueofafterrow)}')
                estimatedvalue = float(valueofafterrow) - std # the estimated value of record 1 = record 2 - std
        
        elif ind ==  len(bookingidx)-1: # if the null record is THE LAST VALUE OF THE BOOKINGID
            print(f'the previous value is {valueofpreviousrow}') 
            estimatedvalue = valueofpreviousrow + std # the estimated value of the last record = second last record + std
        
        elif bookingidx[bookingidx.index.values == afterrowindex]['gyro_x'].empty or bookingidx[bookingidx.index.values == afterrowindex]['gyro_x'].isnull().bool(): # if there are 2 consecutive null values and arent the first or last record
            print(f'the after value is {float(valueofafterrow)}')
            estimatedvalue = valueofpreviousrow + std # estimated value = previous record + std
        
        else:
            print(f'the previous value is {valueofpreviousrow} and the after value is {float(valueofafterrow)}')
            estimatedvalue = np.median([valueofpreviousrow,float(valueofafterrow)]) # the estimated value = median of previous record and next record
        print(f'estimated value is {estimatedvalue}')
        realindex =  set4_1fake[set4_1fake['ID'] == int(id)].index.values # to get the index of the record IN THE SET
        bookingidx.at[int(ind),'gyro_x'] = estimatedvalue # to assign estimated value to the corresponding record IN THE CURRENT BOOKING DATAFRAME THE CODE IS WORKING WITH NOW
        set4_1fake.at[int(realindex),'gyro_x'] = estimatedvalue # this is to assign estimated value to the corresponding record IN THE SET
        ss = set4_1fake[set4_1fake.index.values == int(realindex)]['gyro_x'] # just for printing to debug
        zz = bookingidx[bookingidx.index.values == int(realindex)]['gyro_x'] # just for printing to debug
        print(f'the value back in set4_1fake is now {ss}')
        print(f'the value back in bookingidx is now {zz}')
        print('-----------------------')

bookingid: 34359738383.0, index: 59, id: 12869
the previous value is 0.004470825 and the after value is 0.03767395
estimated value is 0.021072387499999998
the value back in set4_1fake is now 335    0.021072
Name: gyro_x, dtype: float64
the value back in bookingidx is now Series([], Name: gyro_x, dtype: float64)
-----------------------
bookingid: 25769803866.0, index: 39, id: 13278
the previous value is 0.01993864 and the after value is 0.024825562
estimated value is 0.022382101
the value back in set4_1fake is now 344    0.022382
Name: gyro_x, dtype: float64
the value back in bookingidx is now Series([], Name: gyro_x, dtype: float64)
-----------------------
bookingid: 25769803866.0, index: 49, id: 5721757
the previous value is 0.03276681 and the after value is -0.00693943
estimated value is 0.01291369
the value back in set4_1fake is now 143907    0.012914
Name: gyro_x, dtype: float64
the value back in bookingidx is now Series([], Name: gyro_x, dtype: float64)
-----------------------
boo

In [ ]:
set4_1fake.isnull().sum(axis = 0)

ID                   0
bookingID            0
Accuracy          1105
Bearing           1493
acceleration_x       0
acceleration_y       0
acceleration_z       0
gyro_x               0
gyro_y            1059
gyro_z            1258
second             908
Speed                0
dtype: int64

In [ ]:
# THIS IS ONLY FOR SET 3_1fake gyro_y
justnullsdf = set4_1fake[set4_1fake['gyro_y'].isnull()]
bookingids = justnullsdf['bookingID'].unique() # get all the bookingids that have null values inside them
for bookingid in bookingids: #loop for each bookingid so that we are dealing with the null values bookingid by bookingid which is easier to work with
    bookingidx = set4_1fake[set4_1fake['bookingID']==bookingid] # making a datfarame with bookingID = x
    bookingidx = bookingidx.sort_values(by=['second']) # sorting the records by seconds so that it is easier to plot time against gyro_y
    bookingidx = bookingidx.reset_index(drop=True) # reindexing the dataframe for easier coding
    cleanedSetbookingidx = bookingidx.dropna()
    if len(cleanedSetbookingidx) < 5:
        set4_1fake = set4_1fake.drop(set4_1fake[set4_1fake.bookingID == bookingid].index)
        continue
    std = np.std(bookingidx['gyro_y']) # finding the standard deviation of gyro_y
    nulldf = bookingidx[bookingidx['gyro_y'].isnull()] # make dataframe of records with gyro_y being a null value
    for ind in nulldf.index: #retrieve the index of the row with gyro_y = NULL
        id = nulldf[nulldf.index.values == ind]['ID'] # get the value of 'ID' column for that specific null record
        previousrowindex = ind - 1 # this is to retrieve the index of the previous row 
        afterrowindex = ind + 1 # this is to retrieve the index of the next row  
        valueofpreviousrow = bookingidx.iloc[previousrowindex, 8] # this is to retrieve the acceleration of the previous row
        valueofafterrow = bookingidx[bookingidx.index.values == afterrowindex]['gyro_y'] # this is to retrieve the acceleration of the next row
        print(f'bookingid: {bookingid}, index: {int(ind)}, id: {int(id)}')
        if ind == 0: # if the null record is the first record of xbookingid
            if bookingidx[bookingidx.index.values == afterrowindex]['gyro_y'].empty or bookingidx[bookingidx.index.values == afterrowindex]['gyro_y'].isnull().bool(): # if the next record is also a null value, this is for records that are consecutive AT THE START OF THE BOOKINGID
                print(f'the after value is {float(valueofafterrow)}')
                estimatedvalue = float(bookingidx[bookingidx.index.values == afterrowindex+1]['gyro_y']) - std -std # to get estimatedvalue of record 1, record 3- std -std = record 1
            else: # if then next record IS NOT a null value
                print(f'the after value is {float(valueofafterrow)}')
                estimatedvalue = float(valueofafterrow) - std # the estimated value of record 1 = record 2 - std
        
        elif ind ==  len(bookingidx)-1: # if the null record is THE LAST VALUE OF THE BOOKINGID
            print(f'the previous value is {valueofpreviousrow}') 
            estimatedvalue = valueofpreviousrow + std # the estimated value of the last record = second last record + std
        
        elif bookingidx[bookingidx.index.values == afterrowindex]['gyro_y'].empty or bookingidx[bookingidx.index.values == afterrowindex]['gyro_y'].isnull().bool(): # if there are 2 consecutive null values and arent the first or last record
            print(f'the after value is {float(valueofafterrow)}')
            estimatedvalue = valueofpreviousrow + std # estimated value = previous record + std
        
        else:
            print(f'the previous value is {valueofpreviousrow} and the after value is {float(valueofafterrow)}')
            estimatedvalue = np.median([valueofpreviousrow,float(valueofafterrow)]) # the estimated value = median of previous record and next record
        print(f'estimated value is {estimatedvalue}')
        realindex =  set4_1fake[set4_1fake['ID'] == int(id)].index.values # to get the index of the record IN THE SET
        bookingidx.at[int(ind),'gyro_y'] = estimatedvalue # to assign estimated value to the corresponding record IN THE CURRENT BOOKING DATAFRAME THE CODE IS WORKING WITH NOW
        set4_1fake.at[int(realindex),'gyro_y'] = estimatedvalue # this is to assign estimated value to the corresponding record IN THE SET
        ss = set4_1fake[set4_1fake.index.values == int(realindex)]['gyro_y'] # just for printing to debug
        zz = bookingidx[bookingidx.index.values == int(realindex)]['gyro_y'] # just for printing to debug
        print(f'the value back in set4_1fake is now {ss}')
        print(f'the value back in bookingidx is now {zz}')
        print('-----------------------')

bookingid: 515396075587.0, index: 23, id: 7601
the previous value is -0.11611383 and the after value is -0.05965481
estimated value is -0.08788432
the value back in set4_1fake is now 191   -0.087884
Name: gyro_y, dtype: float64
the value back in bookingidx is now Series([], Name: gyro_y, dtype: float64)
-----------------------
bookingid: 25769803804.0, index: 5, id: 13481
the previous value is 0.0014953613 and the after value is -0.012939453
estimated value is -0.00572204585
the value back in set4_1fake is now 348   -0.005722
Name: gyro_y, dtype: float64
the value back in bookingidx is now Series([], Name: gyro_y, dtype: float64)
-----------------------
bookingid: 34359738532.0, index: 27, id: 26757
the previous value is -0.09407325 and the after value is 0.0036651915999999
estimated value is -0.04520402920000005
the value back in set4_1fake is now 648   -0.045204
Name: gyro_y, dtype: float64
the value back in bookingidx is now Series([], Name: gyro_y, dtype: float64)
-----------------

In [ ]:
set4_1fake.isnull().sum(axis = 0)

ID                   0
bookingID            0
Accuracy          1105
Bearing           1492
acceleration_x       0
acceleration_y       0
acceleration_z       0
gyro_x               0
gyro_y               0
gyro_z            1258
second             907
Speed                0
dtype: int64

In [ ]:
# THIS IS ONLY FOR SET 3_1fake gyro_z
justnullsdf = set4_1fake[set4_1fake['gyro_z'].isnull()]
bookingids = justnullsdf['bookingID'].unique() # get all the bookingids that have null values inside them
for bookingid in bookingids: #loop for each bookingid so that we are dealing with the null values bookingid by bookingid which is easier to work with
    bookingidx = set4_1fake[set4_1fake['bookingID']==bookingid] # making a datfarame with bookingID = x
    bookingidx = bookingidx.sort_values(by=['second']) # sorting the records by seconds so that it is easier to plot time against gyro_z
    bookingidx = bookingidx.reset_index(drop=True) # reindexing the dataframe for easier coding
    cleanedSetbookingidx = bookingidx.dropna()
    if len(cleanedSetbookingidx) < 5:
        set4_1fake = set4_1fake.drop(set4_1fake[set4_1fake.bookingID == bookingid].index)
        continue
    std = np.std(bookingidx['gyro_z']) # finding the standard deviation of gyro_z
    nulldf = bookingidx[bookingidx['gyro_z'].isnull()] # make dataframe of records with gyro_z being a null value
    for ind in nulldf.index: #retrieve the index of the row with gyro_z = NULL
        id = nulldf[nulldf.index.values == ind]['ID'] # get the value of 'ID' column for that specific null record
        previousrowindex = ind - 1 # this is to retrieve the index of the previous row 
        afterrowindex = ind + 1 # this is to retrieve the index of the next row  
        valueofpreviousrow = bookingidx.iloc[previousrowindex, 9] # this is to retrieve the acceleration of the previous row
        valueofafterrow = bookingidx[bookingidx.index.values == afterrowindex]['gyro_z'] # this is to retrieve the acceleration of the next row
        print(f'bookingid: {bookingid}, index: {int(ind)}, id: {int(id)}')
        if ind == 0: # if the null record is the first record of xbookingid
            if bookingidx[bookingidx.index.values == afterrowindex]['gyro_z'].empty or bookingidx[bookingidx.index.values == afterrowindex]['gyro_z'].isnull().bool(): # if the next record is also a null value, this is for records that are consecutive AT THE START OF THE BOOKINGID
                print(f'the after value is {float(valueofafterrow)}')
                estimatedvalue = float(bookingidx[bookingidx.index.values == afterrowindex+1]['gyro_z']) - std -std # to get estimatedvalue of record 1, record 3- std -std = record 1
            else: # if then next record IS NOT a null value
                print(f'the after value is {float(valueofafterrow)}')
                estimatedvalue = float(valueofafterrow) - std # the estimated value of record 1 = record 2 - std
        
        elif ind ==  len(bookingidx)-1: # if the null record is THE LAST VALUE OF THE BOOKINGID
            print(f'the previous value is {valueofpreviousrow}') 
            estimatedvalue = valueofpreviousrow + std # the estimated value of the last record = second last record + std
        
        elif bookingidx[bookingidx.index.values == afterrowindex]['gyro_z'].empty or bookingidx[bookingidx.index.values == afterrowindex]['gyro_z'].isnull().bool(): # if there are 2 consecutive null values and arent the first or last record
            print(f'the after value is {float(valueofafterrow)}')
            estimatedvalue = valueofpreviousrow + std # estimated value = previous record + std
        
        else:
            print(f'the previous value is {valueofpreviousrow} and the after value is {float(valueofafterrow)}')
            estimatedvalue = np.median([valueofpreviousrow,float(valueofafterrow)]) # the estimated value = median of previous record and next record
        print(f'estimated value is {estimatedvalue}')
        realindex =  set4_1fake[set4_1fake['ID'] == int(id)].index.values # to get the index of the record IN THE SET
        bookingidx.at[int(ind),'gyro_z'] = estimatedvalue # to assign estimated value to the corresponding record IN THE CURRENT BOOKING DATAFRAME THE CODE IS WORKING WITH NOW
        set4_1fake.at[int(realindex),'gyro_z'] = estimatedvalue # this is to assign estimated value to the corresponding record IN THE SET
        ss = set4_1fake[set4_1fake.index.values == int(realindex)]['gyro_z'] # just for printing to debug
        zz = bookingidx[bookingidx.index.values == int(realindex)]['gyro_z'] # just for printing to debug
        print(f'the value back in set4_1fake is now {ss}')
        print(f'the value back in bookingidx is now {zz}')
        print('-----------------------')

bookingid: 163208757252.0, index: 13, id: 1948964
the previous value is 0.030059814 and the after value is 0.038223267
estimated value is 0.0341415405
the value back in set4_1fake is now 49250    0.034142
Name: gyro_z, dtype: float64
the value back in bookingidx is now Series([], Name: gyro_z, dtype: float64)
-----------------------
bookingid: 163208757252.0, index: 51, id: 966
the previous value is 0.0279235839999999 and the after value is -0.014328003
estimated value is 0.00679779049999995
the value back in set4_1fake is now 25    0.006798
Name: gyro_z, dtype: float64
the value back in bookingidx is now 25    0.031128
Name: gyro_z, dtype: float64
-----------------------
bookingid: 360777252917.0, index: 53, id: 7606
the previous value is -0.0515472 and the after value is 0.16880335
estimated value is 0.058628074999999995
the value back in set4_1fake is now 192    0.058628
Name: gyro_z, dtype: float64
the value back in bookingidx is now Series([], Name: gyro_z, dtype: float64)
-------

In [ ]:
set4_1fake.isnull().sum(axis = 0)

ID                   0
bookingID            0
Accuracy          1104
Bearing           1492
acceleration_x       0
acceleration_y       0
acceleration_z       0
gyro_x               0
gyro_y               0
gyro_z               0
second             906
Speed                0
dtype: int64

In [ ]:
set4_1fake.to_csv('set4_1done.csv')